## Installing the depedencies

In [7]:
# %pip install -qU langchain-ollama
# %pip install -qU langchain-community faiss-cpu
# !python -m pip install -qU langchain-groq --user

In [6]:
import os
from uuid import uuid4
import getpass
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

In [31]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;
- metadata: a dict containing arbitrary metadata.

Here is the link to index of various embedding stores : https://python.langchain.com/v0.2/docs/integrations/vectorstores/

In [3]:
# Defining a list of Document objects
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

## Loading the Embedding Model
When using Ollama embeddings with the langchain_ollama integration, you first need to download the model locally. Here's how it works:

1. Download the Model: You must have the Ollama model downloaded locally. Ollama provides language models that are accessed locally, meaning the model files reside on your machine rather than being fetched from a remote server during runtime.

2. Use langchain_ollama: Once the model is downloaded, you can then load it using the langchain_ollama integration to generate embeddings or perform other operations.

This approach is common for ensuring that the model runs efficiently and offline, with all computations being performed locally. You need to manage the initial setup (like downloading the model) before integrating it with LangChain or other libraries.

In [11]:
embedding_model = OllamaEmbeddings(
    model="llama3",
)

FAISS is primarily in-memory by default. The index is stored in RAM, which allows for fast similarity search and retrieval operations. However, FAISS can also handle larger datasets that don’t fit entirely in memory by using techniques like on-disk indices.

Key points:

- In-memory: By default, FAISS stores indices in memory for high-speed retrieval.
- On-disk storage: FAISS provides options to save indices to disk and reload them later, enabling the handling of larger datasets beyond memory constraints.

In [15]:
# Creating an index 
index = faiss.IndexFlatL2(len(embedding_model.embed_query("hello world"))) # Length = 4096

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [22]:
# Generating unique ids for the documents
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['a6b9696b-5ed4-4302-96ae-32b390cf7ba7',
 '35d2e53c-00cb-431b-a932-7efe11198cd6',
 '47a7be51-d0f6-4930-a31f-8a0c39667dc5',
 '09df7b0c-ff71-42e6-8d10-f6aabf8236eb',
 'ea154ec7-adde-4fb9-8d86-af01f37eb89f']

VectorStore and Runnable: In LangChain, VectorStore objects (used to store and search for vectors, like embeddings) do not automatically work with the LangChain Expression Language (LCEL) because they don't have the necessary methods to "run" as part of a chain.

Retrievers as Runnables: Retrievers are special components in LangChain that can be easily connected into a chain because they follow a standard set of methods (like calling them synchronously, asynchronously, or in batches). These methods make them compatible with LCEL chains.

Creating a Runnable for VectorStore: Even though VectorStore isn’t directly runnable, you can create a simple wrapper (a runnable) around it by deciding how you want to retrieve documents (like using the similarity_search method). This allows you to use VectorStore in LCEL chains without needing to fully subclass or extend it.

In [30]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

Further we can put things into a chain along with the Large Language Model